In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ratings = pd.read_csv('rating_1to5.csv') #Ratings user and place wise
df_rest_info = pd.read_csv('Our_Rest_Info.csv') #Restaurants info

In [3]:
df_ratings.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,4,4,4
1,U1077,135038,5,5,3
2,U1077,132825,4,4,5
3,U1077,135060,3,4,4
4,U1068,135104,3,2,4


In [4]:
df_rest_info.head()

,ID,Cuisine,Longitude,Latitude,Start_Time,End_Time,Avg_Ratings,Budget
0,135110,Fast_Food,-100.9123,18.0340,8,23,1,low
1,135109,Fast_Food,-99.8492,19.1187,8,23,3,low
2,135107,Fast_Food,-99.3568,22.4576,8,23,4,medium
3,135106,Fast_Food,-100.3291,22.1878,8,23,0,high
4,135105,Fast_Food,-99.4616,20.1147,8,23,3,medium


In [5]:
popular_rest = list(set(df_ratings.groupby('placeID').count().query('userID > 11').index))

In [6]:
df_less_restaurants = df_ratings[df_ratings.placeID.isin(popular_rest)]

In [7]:
df_less_restaurants.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,4,4,4
1,U1077,135038,5,5,3
2,U1077,132825,4,4,5
3,U1077,135060,3,4,4
62,U1083,132723,3,2,2


In [8]:
from scipy.sparse import csr_matrix

df_pivoted = df_less_restaurants.pivot(
    index='placeID',
    columns='userID',
    values='rating'
).fillna(0)


mapper_idx = {
    placeID: i for i, placeID in 
    enumerate(list(df_pivoted.index.values))
}

df_sparse_matrix = csr_matrix( df_pivoted.values )

In [9]:
df_pivoted.head()

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1124,U1125,U1126,U1128,U1132,U1134,U1135,U1136,U1137,U1138
placeID,,,,,,,,,,,,,,,,,,,,,
132572,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
132723,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
132754,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
132825,4.0,4.0,4.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,4.0,3.0,1.0,0.0,4.0,0.0
132830,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.neighbors import NearestNeighbors
#make an object for the NearestNeighbors Class.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit the dataset
model_knn.fit(df_sparse_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=20, p=2, radius=1.0)

In [11]:
def matching_ID(mapper, fav):

    """
    return the index of a restaurant based on its ID
    
    Parameters
    ----------    
    mapper: dict, map placeID to the index of that restaurant in the data
    fav: str, ID of the desired restaurant
    
    Return
    ------
    index against an ID
    
    """
    
    match_tuple = []
    for i, idx in mapper.items():
        if i==fav:
            match_tuple.append((i, idx))

    if not match_tuple:
        print('Oops! No match is found')
        return None

    return match_tuple[0][1]

In [12]:
def make_recommendation(model_knn, data, mapper, fav_restaurant, n_recommendations):
    
    """
    return the recommended restaurants against a restaurant
    
    Parameters
    ----------    
    model_knn: object, nearest neighbour model object to fit the data
    data: matrix, sparse data matrix
    mapper: dict, map placeID to the index of that restaurant in the data
    fav_restaurant: str, ID of the desired restaurant
    n_recommendations: int, total number of restaurants needed for that particular restaurants ID
    
    Return
    ------
    index against an ID
    
    """
    
    # fit
    model_knn.fit(data)

    print('You have input restaurant:', fav_restaurant)
    print('This restaurants cuisine are {} \n'.format(df_rest_info[df_rest_info['ID']==fav_restaurant]['Cuisine'].values))

    #Get index against restaurants ID
    idx = matching_ID(mapper, fav_restaurant)
    
    if idx is None:
        return
    
    print('Recommendation system start to make inference')
    print('......\n')
    
    sim, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), sim.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    
    # print recommendations
    print('Recommendations for {}:'.format(fav_restaurant))
    for i, (idx, s) in enumerate(raw_recommends):
        print('{0}: {1} with similarity of {2}%'.format(i+1, reverse_mapper[idx] , round(s*100,2)))
        print('Cusine: {}'.format((df_rest_info[df_rest_info['ID']==reverse_mapper[idx]]['Cuisine'].values)))
        print()
        
            

In [13]:
make_recommendation(model_knn, df_sparse_matrix, mapper_idx, 132921, 5)

You have input restaurant: 132921
This restaurants cuisine are ['Regional'] 

Recommendation system start to make inference
......

Recommendations for 132921:
1: 132572 with similarity of 71.91%
Cusine: ['Desi']

2: 135079 with similarity of 71.39%
Cusine: ['Regional']

3: 132862 with similarity of 70.9%
Cusine: ['Desi']

4: 135058 with similarity of 67.92%
Cusine: ['Regional']

5: 135085 with similarity of 66.44%
Cusine: ['Fast_Food']

